In [1]:
#Preliminary predictive model for clothing predictions

In [2]:
import numpy as np
import pandas as pd
import random
import datetime
from sklearn.model_selection import train_test_split

In [3]:
#Creating the dataset

#features

"""
Weather features:
1. Temperature (Celcius) (-20 to 25)
2. Humidity (%) (relative) (0 to 100)
3. Probability of Precipitation (%) (0 to 100)
4. Wind speed (mph)

User features:
1. Age (7 to 70)
2. Weight (kg)
3. Sex
4. Height (cm)
5. Fat percentage
"""

#output
"""
output will be in the form of a boolean array with each index presenting different clothing articles
0. short sleve shirt + long pants (always on)
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

'\noutput will be in the form of a boolean array with each index presenting different clothing articles\n0. short sleve shirt + long pants (always on)\n1. Thermal (both top and bottom)\n2. Jacket/Hoodie\n3. light downjacket\n4. Thick down/Winter down\n5. Wind breaker\n6. Umbrella/Raincoat\n'

In [4]:
from datetime import date
data = []
currentdate = date.today()
for i in range(100000):
    date = (currentdate - datetime.timedelta(i)).strftime("%d/%m/%Y") #display in day/month/year
    temperature = random.randint(0, 45) - 20
    humidity = random.randint(0, 100)
    precipitation = random.randint(0, 100)
    windspeed = random.randint(0, 40) #mph
    age = random.randint(7, 70)
    weight = random.randint(20, 120) #kg
    height = random.randint(110, 200) #cm
    sex = random.randint(0,1) # 0:M 1:F
    #fat percentage formula from
    #https://www.gaiam.com/blogs/discover/how-to-calculate-your-ideal-body-fat-percentage
    
    BMI = weight/(height**2)*10000
    if sex == 0:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 16.2
        
    elif sex == 1:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 5.4
    
    #adjustment for negative fatpercentage
    if fatpercentage < 0:
        fatpercentage = 1
        
    datapoint = (date, temperature, humidity, precipitation, windspeed, age, weight, height, sex, fatpercentage, BMI)
    data.append(datapoint)
    

In [5]:
columnstring = 'date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi'
column = columnstring.split(',')
df = pd.DataFrame(data, columns = column)

In [6]:
#df[:100]

In [7]:
#layers

"""
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""




#cold resistance calculation

male_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}


female_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}

# cold resistance categories:
# low: 0 - 24
# mid-low: 25 - 49
# mid-high: 50 - 74
# high: 75 - 99
cold_resistance_scores = {
    'low' : 25,
    'mid-low' : 50,
    'mid-high' : 75,
    'high' : 100
}




In [8]:
#returns cold resistance score
#formula:

#cold_resistance = age_score * gender_age_weight + bmi_score + gender_bmi_weight + gender_fatpercentage_weight
def coldresistance(sex, age, fatpercentage, BMI):
    #female case
    if sex == 1:
        if (age <= 12 or age >= 55):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 13 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 14):
            fatpercentage_score = cold_resistance_scores['low']
        elif (14 < fatpercentage <= 20):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (20 < fatpercentage <= 25): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
      
    #men
    else:
        if (age <= 12 or age >= 55):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 13 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 8):
            fatpercentage_score = cold_resistance_scores['low']
        elif (8 < fatpercentage <= 15):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (15 < fatpercentage <= 24): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
            
    #female case
    if sex == 1:
        cold_resistance = age_score/100*female_weights['age'] + bmi_score/100*female_weights['bmi'] + fatpercentage_score/100*female_weights['fatpercentage']
    
    #male case
    elif sex == 0:
        cold_resistance = age_score/100*male_weights['age'] + bmi_score/100*male_weights['bmi'] + fatpercentage_score/100*male_weights['fatpercentage']
    
    return cold_resistance

In [9]:
df['cold_resistance'] = df.apply(lambda row : coldresistance(row['sex'], row['age'], row['fatpercentage'], row['bmi']), axis = 1)

In [10]:
#df[df['cold_resistance'] > 99]

In [11]:
#rules
"""
1. If temp < 5 : Thermal = True and thickdown = True
2. If temp < 15 : Jacket = True
3. If temp < 10 : lightdown = True
4. If windspeed >= 25 : windbreaker = True
5. If precipitation > 50%: umbrella/raincoast = True


cold resistance categories:
low: 0 - 24
mid-low: 25 - 49
mid-high: 50 - 74
high: 75 - 99
"""

clothesmap = {
    'thermal' : 0, #(both top and bottom)
    'hoodie' : 1,
    'light_down' : 2,
    'thick_down' : 3,
    'wind_breaker' : 4,
    'umbrella' : 5
}

def clothingPredictor(coldresistance, temperature, humidity, precipitation, windspeed):
    #prediction is an array of booleans with each index corresponding to a certain clothing article
    prediction = [False] * 6
    
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
    
    
    #low cold resistance
    if coldresistance <= 25:
        #print('low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        elif temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        #if temp > 25 dont put on any additional layers
        else:
            pass
            
    #mid-low cold resistance
    elif 25 < coldresistance <= 50:
        #print('mid- low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 25 dont put on any additional layers
        else:
            pass
        
    #mid-high cold resistance
    elif 50 < coldresistance <= 75:
        #print('mid - high cr')
        if 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 20 dont put on any additional layers
        else:
            pass
        
    #high cold resistance
    elif coldresistance > 75:
        #print('high cr')
        if 10 <= temperature <= 15:
            prediction[clothesmap['hoodie']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif -10 < temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= -10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            #print('yay')
        #if temp > 15 dont put on any additional layers
        else:
            pass
        
    
    
    return prediction

In [12]:
for article in clothesmap:
    df[article] = df.apply(lambda row : clothingPredictor(row['cold_resistance'], row['temperature'], row['humidity'],\
                                                          row['precipitation'], row['windspeed'])[clothesmap[article]], axis = 1)

In [14]:
df.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance,thermal,hoodie,light_down,thick_down,wind_breaker,umbrella
0,11/01/2022,-13,83,54,23,32,21,198,1,8.387916,5.356596,55.0,True,True,True,True,False,True
1,10/01/2022,4,76,43,38,30,31,179,0,2.310125,9.675104,55.0,True,True,True,False,True,False
2,09/01/2022,-20,50,96,40,40,94,110,0,86.223140,77.685950,100.0,True,True,True,True,True,True
3,08/01/2022,-12,56,46,35,44,81,172,1,37.575598,27.379665,90.0,True,True,True,True,True,False
4,07/01/2022,5,21,95,27,12,117,121,0,82.455089,79.912574,70.0,True,True,True,False,True,True


In [15]:
features = df[['temperature', 'humidity', 'precipitation', 'windspeed', 'cold_resistance', 'age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi', 'cold_resistance' ]]

In [17]:
output = df[['thermal', 'hoodie', 'light_down', 'thick_down', 'wind_breaker', 'umbrella']]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features, output, test_size=0.25, random_state=10)

In [20]:
import xgboost as xgb

In [22]:
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, X_train, num_round)

TypeError: ('invalid cache item: DataFrame', [       temperature  humidity  precipitation  windspeed  cold_resistance  age  \
73663           16        66            100         34             70.0   67   
46694          -13        92             27         35             90.0   53   
12163            2        89             67         30             50.0   22   
58752          -18        23             12         34             47.5   62   
49821            7        58             46         21             80.0   22   
...            ...       ...            ...        ...              ...  ...   
9372            -8         7             45         31             47.5   12   
93553          -12        16             63         11             52.5   46   
50496          -19        73             83         21             70.0   58   
94735            5        13             71         34             52.5   45   
83209           17        34             61         26             70.0   